In [1]:
print('hello')


hello


In [2]:
import os
file_name = r"C:\Users\98937\NOBY\carbon_bombs_kuhne_2022.xlsx"
print(os.path.exists(file_name))

True


In [2]:
import xarray as xr
import pickle as pk
import matplotlib as plt
import os
import cartopy as cp
import geopandas as gp
import sys 
import numpy as np
import pandas as pd
import scipy as sp
import regionmask as rg
import textwrap as tp

def pre_processing_carbon_bombs(excel_file):
    df = pd.read_excel(excel_file, sheet_name='Full Carbon Bombs List')
    return df 

file_name = r"C:\Users\98937\NOBY\carbon_bombs_kuhne_2022.xlsx"

print(os.path.exists(file_name))  # should be True

df = pre_processing_carbon_bombs(file_name)

print(df.head())


True
   New                    Name    Country  Potential emissions (Gt CO2)  \
0    *         Tannezuft Shale    Algeria                      2.347867   
1  NaN  Hassi R'Mel (Domestic)    Algeria                      2.271542   
2  NaN          Hassi Messaoud    Algeria                      1.215923   
3  NaN       Vaca Muerta Shale  Argentina                      5.177782   
4  NaN   Red Hill Coal Project  Australia                      4.564469   

      Fuel  Unnamed: 5  Country.1  # of Carbon Bombs  Potential Emissions  \
0  Oil&Gas         NaN        NaN                NaN                  NaN   
1  Oil&Gas         NaN        NaN                NaN                  NaN   
2  Oil&Gas         NaN    Algeria                3.0             5.835333   
3  Oil&Gas         NaN  Argentina                1.0             5.177782   
4     Coal         NaN        NaN                NaN                  NaN   

   Rank           Country.2  # of Carbon Bombs.1  Potential Emissions.1  \
0   1.